In [9]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [10]:
api = KaggleApi()
api.authenticate()

In [11]:
api.competition_download_file("sentiment-analysis-on-movie-reviews", "train.tsv.zip", path="./")

100%|████████████████████████████████████████████████████████████| 1.28M/1.28M [00:00<00:00, 7.97MB/s]

In [2]:
import zipfile
with zipfile.ZipFile("train.tsv.zip", "r") as zipref:
    zipref.extractall("./")

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("train.tsv", sep="\t")

In [6]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [9]:
df.drop_duplicates(subset="SentenceId", keep="first", inplace=True, ignore_index=True)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,64,2,"This quiet , introspective and entertaining in...",4
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
3,117,4,A positively thrilling combination of ethnogra...,3
4,157,5,Aggressive self-glorification and a manipulati...,1


In [8]:
len(df)

8529

#### Encode Data